In [ ]:
class GraphConvolution(Module):
    def _init_(self, in_features, out_features, bias=True):
        super(Graphconvolution, self)._init_()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
            self.reset_parameters()
    def reset_parameters(self):
        stdv = 1. /. math.sqrt(self.weight.size(1))self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj. support)
        if self.bias is not None:
            return output + self.bias
        else:
            réturn output
        def  _repr_(self):
            return self._class_._name_  + "(" + str(self.in_features) + str(self.out_features)+ ")"

In [5]:
import dgl
from dgl.data import citation_graph as citegrh


# 下载并加载 Cora 数据集
dataset_cora = citegrh.load_cora(raw_dir="./dataset/cora/")

g_cora = dataset_cora[0]
num_class_cora = dataset_cora.num_classes
feature_cora = g_cora.ndata['feat']
label_cora = g_cora.ndata['label']

# get data split
train_mask_cora = g_cora.ndata['train_mask']
val_mask_cora = g_cora.ndata['val_mask']
test_mask_cora = g_cora.ndata['test_mask']

print(train_mask_cora.size(), val_mask_cora.size(), test_mask_cora.size())
print(num_class_cora, feature_cora.size(), label_cora.size())
print(g_cora.adjacency_matrix())

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
torch.Size([2708]) torch.Size([2708]) torch.Size([2708])
7 torch.Size([2708, 1433]) torch.Size([2708])
SparseMatrix(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                             [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
             values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             shape=(2708, 2708), nnz=10556)


In [2]:
# 下载并加载 Citeseer 数据集
dataset_citeseer = citegrh.load_citeseer(raw_dir="./dataset/citeseer/")

g_citeseer = dataset_citeseer[0]
num_class_citeseer = dataset_citeseer.num_classes
feature_citeseer = g_citeseer.ndata['feat']
label_citeseer = g_citeseer.ndata['label']

# get data split
train_mask_citeseer = g_citeseer.ndata['train_mask']
val_mask_citeseer = g_citeseer.ndata['val_mask']
test_mask_citeseer = g_citeseer.ndata['test_mask']

print(train_mask_citeseer.size(), val_mask_citeseer.size(), test_mask_citeseer.size())
print(num_class_citeseer, feature_citeseer.size(), label_citeseer.size())

./dataset/citeseer/citeseer.zip: 100%|██████████| 239k/239k [00:00<00:00, 815kB/s] 


Extracting file to ./dataset/citeseer/citeseer_d6836239
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
torch.Size([3327]) torch.Size([3327]) torch.Size([3327])
6 torch.Size([3327, 3703]) torch.Size([3327])


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import function as fn


class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(in_features, out_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        nn.init.xavier_uniform_(self.weight)  # 初始化权重
        nn.init.zeros_(self.bias)  # 初始化偏置

    def forward(self, g, features):
        with g.local_scope():
            g.ndata['h'] = features
            g.update_all(message_func=fn.copy_u('h', 'm'),
                        reduce_func=fn.sum('m', 'h'))
            h = g.ndata['h']
            h = torch.matmul(h, self.weight) + self.bias
            return h
    

class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(in_features, hidden_features)
        self.gc2 = GraphConvolution(hidden_features, out_features)

    def forward(self, g, features):
        h = F.relu(self.gc1(g, features))
        h = self.gc2(g, h)
        return h

In [ ]:
import torch.optim as optim
import optuna


def objective(trial):
    # 定义一个目标函数，用于在 Optuna 中进行优化
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)  # 在对数尺度上搜索学习率

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(g, features)
        loss = criterion(output[train_mask], labels[train_mask])
        loss.backward()
        optimizer.step()

        # 在验证集上计算损失
        model.eval()
        with torch.no_grad():
            val_output = model(g, features)
            val_loss = criterion(val_output[val_mask], labels[val_mask])

        trial.report(val_loss.item(), epoch)  # 将验证集损失报告给 Optuna
        if trial.should_prune():  # 如果性能不佳，剪枝（提前停止）
            raise optuna.TrialPruned()

    return val_loss.item()


def train_model(model, g, features, labels, train_mask, val_mask, epochs=100):
    study = optuna.create_study(direction='minimize')  # 创建一个 Optuna study 实例
    study.optimize(objective, n_trials=100)  # 运行优化器，尝试不同的超参数组合

    print('最佳学习率:', study.best_params['lr'])  # 打印最佳学习率

    # 使用最佳学习率重新训练模型
    lr = study.best_params['lr']
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(g, features)
        loss = criterion(output[train_mask], labels[train_mask])
        loss.backward()
        optimizer.step()

        # 在验证集上计算损失
        model.eval()
        with torch.no_grad():
            val_output = model(g, features)
            val_loss = criterion(val_output[val_mask], labels[val_mask])

        print(
            f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    print('训练完成。')

In [ ]:
import numpy as np

# 划分数据集


def split_dataset(num_nodes, train_ratio=0.1, val_ratio=0.1, test_ratio=0.8):
    indices = np.arange(num_nodes)
    np.random.shuffle(indices)
    train_size = int(num_nodes * train_ratio)
    val_size = int(num_nodes * val_ratio)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size+val_size]
    test_indices = indices[train_size+val_size:]
    train_mask = np.zeros(num_nodes, dtype=bool)
    val_mask = np.zeros(num_nodes, dtype=bool)
    test_mask = np.zeros(num_nodes, dtype=bool)
    train_mask[train_indices] = True
    val_mask[val_indices] = True
    test_mask[test_indices] = True
    return train_mask, val_mask, test_mask


# 划分 Cora 数据集
num_nodes = g.num_nodes()
train_mask, val_mask, test_mask = split_dataset(num_nodes)

print(
    f'Train nodes: {train_mask.sum()}, Val nodes: {val_mask.sum()}, Test nodes: {test_mask.sum()}')

# 训练模型
model = GCN(in_features=features.shape[1],
            hidden_features=16, out_features=labels.max().item() + 1)
train_model(model, g, features, labels, train_mask, val_mask)

In [ ]:
# 划分数据集
def split_edges(g, val_ratio=0.1, test_ratio=0.1):
    num_edges = g.num_edges()
    edges = np.arange(num_edges)
    np.random.shuffle(edges)
    val_size = int(num_edges * val_ratio)
    test_size = int(num_edges * test_ratio)
    val_edges = edges[:val_size]
    test_edges = edges[val_size:val_size+test_size]
    train_edges = edges[val_size+test_size:]
    return train_edges, val_edges, test_edges


# 划分 Cora 数据集的边
train_edges, val_edges, test_edges = split_edges(g)

print(
    f'Train edges: {len(train_edges)}, Val edges: {len(val_edges)}, Test edges: {len(test_edges)}')

# 训练模型
model = GCN(in_features=features.shape[1], hidden_features=16, out_features=1)
train_model(model, g, features, labels, train_edges)

In [ ]:
# 选择最佳超参数组合
best_hidden_size = 16
best_lr = 0.01

# 重新训练模型
model = GCN(in_features=features.shape[1],
            hidden_features=best_hidden_size, out_features=labels.max().item() + 1)
train_model(model, g, features, labels, train_mask, val_mask, lr=best_lr)

# 在测试集上进行最终测试
model.eval()
with torch.no_grad():
    test_output = model(g, features)
    test_loss = criterion(test_output[test_mask], labels[test_mask])

print(f'Final Test Loss: {test_loss.item():.4f}')

# 计算 Top 1 Accuracy
_, predicted = torch.max(test_output[test_mask], 1)
correct = (predicted == labels[test_mask]).sum().item()
total = test_mask.sum().item()
accuracy = correct / total
print(f'Top 1 Accuracy: {accuracy:.4f}')